# Récupération des tweets et articles en rapport au marché boursier US via les API 

**Projet Python - 2A ENSAE** . 

Elena Loumagne / Jérémie Darracq 

## Introduction
Ce notebook a été créé extraire des données via des requêtes envoyées aux API. Pour obtenir les clés d'accès aux API, nous avons eu besoin de créer un compte FinHub et un compte twitter developper.

## Packages utilisés

In [1]:
## Pour la récupération des données

#%pip install finnhub-python
import finnhub
import pandas as pd 
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

## Pour le preprocessing

#%pip install demoji
#%pip install nltk

import demoji
import re
from nltk.corpus import stopwords
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')






c:\Users\elelo\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elelo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\elelo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\elelo\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\elelo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Récupération des article via l'API de Finhub
Dans cette section, on définit les fonctions permettant la connexion à l'API finhub et la récupération des données selon les termes clés, les dates et d'autres caractéristiques.

In [2]:
## Fonction qui transforme les données fournit par l'API en un dictionnaire 

def get_data(article):
    data = {
        'date': time.strftime("%D %H:%M", time.localtime(article['datetime'])),
        'headline': article['headline'],
        'company': article['related'],
        'abstract': article['summary'],
        'source': article['source']
    }
    return data


In [3]:

finnhub_client = finnhub.Client(api_key="ce74bdiad3iakcsvp120ce74bdiad3iakcsvp12g")  ## Fonction d'appel à l'API

start_date = "2022-10-01" ## Date du début de la requête 
end_date = "2022-12-05"  ## Date de fin de la requête 

## Liste des companies du SP500 apparaissant dans les articles 

List_company = ['AMZN','AAPL','MSFT','META','JPM','JNJ','GOOGL','PFE','BAC','NFLX','MA','MCD','GS','INTC','TMUS','WMT','CBOE','MRK','WFC','BA','MRNA','NDAQ','NKE','SLB','TSLA','VZ','T','AXP','BRK.B','CAT','CVX','KO','EA','FDX','GE','IBM','UNH','XOM','NVDA','PEP','COST','DIS','PM','ATVI','ADBE','AAL','AIG','BIIB','AVGO','COF']

## Pour chaque companie nous effectuons un appel à l'API 

df = pd.DataFrame() 
for symbol in List_company :
    response = finnhub_client.company_news(symbol, _from= start_date, to=end_date)
    for article in response:
        row = get_data(article) 
        df = df.append(row, ignore_index=True)

## On stock la base de donné fournit par Finhub 

df.to_csv("Data/data_finhub.csv",index=False)



In [4]:
## On modifie le format de la date 
df["date"]=df["date"].apply(lambda x : x[0:8])

## Data preprocessing des articles Finhub 

In [5]:
## fonction qui nettoie le texte

def transform(texte):
    texte = texte.lower() # mettre les mots en minuscule
    for item in re.compile("([#]\w+)").findall(texte):
        texte=texte.replace(item, "")
    # retirer les apostrophes '
    for item in re.compile("([\’])").findall(texte):
        texte=texte.replace(item, " ")
    for item in re.compile("([\'])").findall(texte):
        texte=texte.replace(item, " ")
    # retirer les points de suspension
    for item in re.compile("([.]{1,5})").findall(texte):
        texte=texte.replace(item, "")
    texte = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", texte)
    return texte

In [6]:
## On nettoie les résumés des articles 
df["abstract_clean"]=df["abstract"].apply(lambda x : transform(x))


In [7]:
##  fonction qui supprime les petits mots non pertinent pour l'analse ( ex: you,the )
stopwords = nltk.corpus.stopwords.words('english')
words = set(nltk.corpus.words.words())

def retrait_sw(text):
    return ' '.join([word for word in text.split() if word.casefold() not in stopwords ])

df["abstract_clean"] = df["abstract_clean"].apply(retrait_sw)


In [8]:
## On "stem" les résumés des articles pour ne garder que la racine des mots 
stemmer = SnowballStemmer(language="english")
df["abstract_stemmed"]=df["abstract_clean"].apply(lambda x : ' '.join([stemmer.stem(word) for word in x.split()]))
df

,date,headline,company,abstract,source,abstract_clean,abstract_stemmed
0,12/05/22,"Why Amazon, Okta, and Roku Stocks All Slumped ...",AMZN,A report on the state of the service sector fu...,Yahoo,report state service sector fueled fears feder...,report state servic sector fuel fear feder res...
1,12/05/22,Apple and Amazon resume Twitter advertising,AMZN,Yahoo Finance tech editor Dan Howley outlines ...,Yahoo,yahoo finance tech editor dan howley outlines ...,yahoo financ tech editor dan howley outlin app...
2,12/05/22,"Amazon's (AMZN) AWS Gets Selected by Yahoo, Bo...",AMZN,Amazon's (AMZN) AWS is chosen by Yahoo as the ...,Yahoo,amazon (amzn) aws chosen yahoo preferred publi...,amazon (amzn) aw chosen yahoo prefer public cl...
3,12/05/22,Amazon.com Inc. stock underperforms Monday whe...,AMZN,Shares of Amazon.com Inc. shed 3.31% to $91.01...,MarketWatch,"shares amazoncom inc shed $ monday, proved all...","share amazoncom inc shed $ monday, prove all-a..."
4,12/05/22,"After explosive growth, Philadelphia tech star...",AMZN,"Triumph Technology Solutions, which has experi...",Yahoo,"triumph technology solutions, experienced year...","triumph technolog solutions, experienc year-ov..."
...,...,...,...,...,...,...,...
10833,10/06/22,Former Amazon cloud engineer gets probation fo...,COF,"The hack compromised about 140,000 Social Secu...",Yahoo,"hack compromised , social security numbers , b...","hack compromis , social secur number , bank ac..."
10834,10/05/22,Capital One Financial Corporation to Webcast C...,COF,"On Thursday, October 27, 2022, at approximatel...",Yahoo,"thursday, october , , approximately : pm easte...","thursday, octob , , approxim : pm eastern time..."
10835,10/04/22,Why Is RIVN Stock Up Today? Rivian Confirmed 2...,COF,RIVN stock is up after Rivian announced it pro...,InvestorPlace,"rivn stock rivian announced produced , vehicle...","rivn stock rivian announc produc , vehicl trac..."
10836,10/03/22,"Q3 2022 Bank Analysts' Top 3 Picks, Earnings E...",COF,"Big bank stocks are down 27% on average, but a...",SeekingAlpha,"big bank stocks average, analysts see sunny da...","big bank stock average, analyst see sunni day ..."


Nous avons récupéré et nettoyé tous les abstracts des articles. Nous pouvons dès à présent les stocker dans un fichier .csv pour pouvoir les analyser.

In [9]:
df.to_csv("Data/data_finhub.csv",index=False)

## Récupération des article via l'API de Twitter
Dans cette section, on définit les fonctions permettant la connexion à l'API twitter et la récupération des données selon les termes clés, les dates et d'autres caractéristiques.

Pour obtenir un maximum de tweet, nous avons dû faire la demande pour mettre à jour notre compte twitter developper en un compte elevated. Cela permet d'obtenir un nombre de tweets plus important.  
Malheureseument, la période d'extraction des tweets avec l'API ne s'étend qu'aux 7 derniers jours. C'est pourquoi nous utiliserons dans un second temps les tweets pour la construction de notre modèle de prédiction.

In [11]:
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAEHijwEAAAAA5k533gmjemyLZvGcHJ85KptB2ag%3DakjvNX75aeG15S5hKt8tVPniNlrXN0DihoVURgMjmoXXcr7e6M"

In [12]:
def get_data(tweet):
    data = {
        'date': tweet['created_at'],
        'text': tweet['text']
    }
    return data

**ATTENTION :** le code pour récupérer les tweets financiers des 7 derniers jours prend un dizaine de minutes à tourner. Si vous souhaitez vous épargner ce temps d'attente, vous pouvez directement passer à l'étape suivante en important la base de tweets non nettoyée.

In [13]:
# Appel à l'API
endpoint = 'https://api.twitter.com/2/tweets/search/recent'
headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
params = {
    'query': '(VIX OR S&P 500 OR CBOE OR investement OR stock market OR Federal Reserve Bank OR stock price OR inflation OR bonds ) (lang:en)', # mots présents dans les tweets
    'max_results': '100', #  on récupère 100 tweets à chaque appel à l'API. C'est le nombre maximal de tweets que l'on peut obtenir avec notre compte.
    'tweet.fields': 'created_at,lang'
}


dtformat = '%Y-%m-%dT%H:%M:%S.000Z'  # le format de la date recquis par l'API Twitter

def time_travel(now, mins):
    '''
    Permet de modifier la date en voyageant de 'mins' minutes dans le passé 

    'now : str 
        date actuelle
    
    'mins : int
        minutes à soustraire
    '''

    now = datetime.strptime(now, dtformat)
    back_in_time = now - timedelta(minutes=mins)
    return back_in_time.strftime(dtformat)
    
now = datetime.now() # date actuel
last_week = now - timedelta(days=7)  # date de fin
now = now.strftime(dtformat)  # convertit la date au format de l'API


now=time_travel(now,60)
df_tweet = pd.DataFrame()  

time.sleep(20) # on attend 20s pour déclencher l'appel à l'API


while True:
    if datetime.strptime(now, dtformat) < last_week:
        # si on atteint les 7 jours, on sort de la boucle
        break
    pre60 = time_travel(now, 30)  
    
    params['start_time'] = pre60
    params['end_time'] = now
    response = requests.get(endpoint,
                            params=params,
                            headers=headers)  # envoie de la requête
    now = pre60  # on voyage 60 min avant pour récupérer un maximum de tweet sur chaque heure

    # on ajoute les tweets à notre df
    for tweet in response.json()['data']:
        row = get_data(tweet)  
        df_tweet = df_tweet.append(row, ignore_index=True)

df_tweet.to_csv("Data/tweet_not_cleaned.csv",index=False) # enregistrement des données

## Data preprocessing des Tweets

**Import des tweets pas nettoyées :**

In [15]:
df_tweet=pd.read_csv("Data/tweet_not_cleaned.csv")

In [20]:
## on supprime tous les pseudos qui comportent le mot vix car cela ne correspond pas aux tweet sur la finance
for index,row in df_tweet.iterrows():
    tweet=row["text"]
    real_pseudo=[]
    
    for word in tweet.split():
        if '@' in word:
            real_pseudo.append(word)

    vix_in_pseudo=["vix" in pseudo.lower() for pseudo in real_pseudo]

    if True in vix_in_pseudo:
        df_tweet.drop(index, inplace=True)

df_tweet.reset_index(inplace=True,drop=True)

## on modifie la date 
df_tweet["date"]=df_tweet["date"].apply(lambda x : x[0:10])

In [27]:
## Fonction qui nettoie les tweets

def transform_tweet(texte):
    texte = texte.lower() # mettre les mots en minuscule
    #retirer les liens
    for item in re.compile("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+").findall(texte):
        texte=texte.replace(item, "")
    # enlever le retour à la ligne
    texte = texte.replace("\n", " ").replace("\r", "") 
    # supprimer ",", "!", "?", "%", "(",")","/",'"', "$","£", "_", "-", "+", "*", "µ", ":","&,"§" 
    texte = re.sub(r"[,\!\?\%\(\)\/\"\$\£\+\*\µ,\:\&\§]", " ", texte) 
    # retirer les hashtags #
    for item in re.compile("([#]\w+)").findall(texte):
        texte=texte.replace(item, "")
    # retirer les apostrophes '
    for item in re.compile("([\’])").findall(texte):
        texte=texte.replace(item, " ")
    for item in re.compile("([\'])").findall(texte):
        texte=texte.replace(item, " ")
    # retirer les points de suspension
    for item in re.compile("([.]{1,5})").findall(texte):
        texte=texte.replace(item, "") 
    # retirer les personnes tagées
    for item in re.compile("([@]\w+)").findall(texte):
        texte=texte.replace(item, "")
    # retirer les adresses mail
    for item in re.findall('\S+@\S+', texte) :
        texte=texte.replace(item, "")
    # retire les mots contenant des chiffres
    texte = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", texte)
    # retirer les emojis
    for item in demoji.findall(texte):
        texte=texte.replace(item,"")
    return texte

df_tweet["tweet_clean"]=df_tweet["text"].apply(lambda x : transform_tweet(x))

## on retire les stopswords

df_tweet["tweet_clean"] = df_tweet["tweet_clean"].apply(retrait_sw)

In [28]:
## on "stem" les tweets
df_tweet["tweet_stemmed"]=df_tweet["tweet_clean"].apply(lambda x : ' '.join([stemmer.stem(word) for word in x.split()]))

Nous avons dès à présent finit le nettoyage des tweets. Nous les enregistrons dans le fichier .csv pour pouvoir les utiliser.

In [29]:
df_tweet.to_csv("Data/tweets.csv",index=False)